In [1]:
from voice import *
from voiceEN import *

C:\Users\shiha\anaconda3\envs\lang\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
C:\Users\shiha\AppData\Roaming\Python\Python311\site-packages\TTS\api.py:77: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")
C:\Users\shiha\anaconda3\envs\lang\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
import moviepy.editor as mpy
from moviepy.audio.AudioClip import AudioArrayClip

def audio_fadein(clips,fade_in = 0.05):
    final_clips = [
        clip.audio_fadein(fade_in).audio_fadeout(fade_in)
        for clip in clips
    ]
    return(final_clips)

def text2audio(text:str, lang:str, audio = "./cache/audio/1.wav"):
    #audio from text
    text = text.replace('\r', ' ').replace('\n', ' ')
    if lang == "en":
        tts.tts_to_file(text=text, file_path=audio)
    else:
        o3, o4 = tts_fn(text, lang = lang)
        from scipy.io.wavfile import write
        write(audio,o4[0],o4[1])
    return()

def text2video(text:str, lang:str, title = "      ", folder = "./cache/", prefix = "1"):
    #make audio
    o4 = text2audio(text, lang = lang, audio = folder+"audio/"+prefix+".wav");
    audio = mpy.AudioFileClip(folder+"audio/"+prefix+".wav")
    #audio = AudioArrayClip(o4[1],fps = o4[0])
    #make text
    if lang == "cn":
        txt_clip  = mpy.TextClip(text, fontsize = 20, color = "white", font = "Microsoft-YaHei-&-Microsoft-YaHei-UI")
    else:
        txt_clip  = mpy.TextClip(text, fontsize = 20, color = "white")
    txt_clip = txt_clip.set_pos('bottom').set_duration(audio.duration)
    title_clip  = mpy.TextClip(title, fontsize = 40, color = "black")
    title_clip = title_clip.set_pos('top').set_duration(audio.duration)
    #make video
    clip = mpy.ImageClip("./cache/img.png")
    clip = clip.set_duration(audio.duration)
    clip.audio = audio
    clip  = mpy.CompositeVideoClip([clip, txt_clip, title_clip])
    clip.fps = 12
    #clip.write_videofile("./cache/clip.mp4");
    return(clip)

def json2video(json:list, lang = "cn", output = "./cache/video.mp4"):
    videos = []
    count = 0;
    for l in json:
        if l['cg'] != "":
            #get cg
            import wget
            wget.download(l['cg'], out="./cache/img.png")
            #split text
            text = l['summary']
            split = text.replace('。','.').split('.')
            clips = []
            for s in split:
                if len(s) > 5:
                    count = count + 1;
                    clips.append(text2video(text = s, lang = lang, title = l['stage'], prefix = str(count)))
            clips = audio_fadein(clips)
            videos.append(mpy.concatenate_videoclips(clips, method="compose"))
    videos = audio_fadein(videos)
    concat_clip = mpy.concatenate_videoclips(videos, method="compose")
    concat_clip.write_videofile(output)

#output chapter content to html
def run(query:str, output = "./cache/video.mp4", path="../db/cndb", lang = "cn"):
    import chromadb
    client = chromadb.PersistentClient(path=path)
    db = client.get_collection("langchain")
    data = db.get()
    tmp = []
    for k in range(len(data["ids"])):
        if query in data["metadatas"][k]["stage"]:
            this = {"stage": data["metadatas"][k]["stage"], 
                    "summary": data["documents"][k],
                    "cg": data["metadatas"][k]["cg"],
                   }
            tmp.append(this);
    json2video(json = tmp, lang = lang, output = output)



In [ ]:
run("13-22", lang = "cn", path="../db/cndb", output = "13-22.mp4")

t:  27%|█████████████████▉                                                | 543/1994 [00:26<01:11, 20.28it/s, now=None]

In [4]:
run("13-22", lang = "en",  path="../db/arkdb")

In [14]:
import chromadb
query = "13-22"
path="../db/cndb"
client = chromadb.PersistentClient(path=path)
db = client.get_collection("langchain")
data = db.get()
tmp = []
for k in range(len(data["ids"])):
    if query in data["metadatas"][k]["stage"]:
        this = {"stage": data["metadatas"][k]["stage"], 
                "summary": data["documents"][k],
                "cg": data["metadatas"][k]["cg"],
               }
        tmp.append(this);



In [34]:
text = tmp[0]['summary']
text.split('。')[0]

'故事的第一部分发生在一个帐篷内，有两个人物在对话'

In [48]:
res = text2audio(text.split('。')[0], lang = 'cn')

In [56]:
AudioArrayClip(res[1], fps=res[0])

TypeError: 'numpy.float32' object is not iterable

In [52]:
clip = text2video(text.split('。')[0],lang='cn',title=tmp[0]['stage'])
clip.write_videofile("./cache/video.mp4");

TypeError: 'numpy.float32' object is not iterable

In [55]:
import numpy as np
from moviepy.audio.AudioClip import AudioArrayClip

rate = 44100  # Sampling rate in samples per second.
duration = 2  # Duration in seconds

data_mono = np.random.uniform(-1, 1, (int(duration*rate/2), 1))

audio_mono = AudioArrayClip(data_mono, fps=rate)


In [54]:
data_mono

array([[-0.74734869],
       [ 0.94559108],
       [-0.10485379],
       ...,
       [-0.52769184],
       [ 0.86502881],
       [-0.35873155]])

In [57]:
res[1]

array([-8.8289869e-07, -2.6668969e-05, -1.7760936e-05, ...,
        2.4428172e-04,  2.3002260e-04,  2.8541948e-05], dtype=float32)